## Import the libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

In [2]:
#store the data
df = pd.read_csv("aapl_csv")
df 

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-12-31,39.632500,39.840000,39.119999,39.435001,38.282612,140014000
1,2019-01-02,38.722500,39.712502,38.557499,39.480000,38.326294,148158800
2,2019-01-03,35.994999,36.430000,35.500000,35.547501,34.508709,365248800
3,2019-01-04,36.132500,37.137501,35.950001,37.064999,35.981865,234428400
4,2019-01-07,37.174999,37.207500,36.474998,36.982498,35.901768,219111200
...,...,...,...,...,...,...,...
500,2020-12-23,132.160004,132.429993,130.779999,130.960007,130.157608,88223700
501,2020-12-24,131.320007,133.460007,131.100006,131.970001,131.161407,54930100
502,2020-12-28,133.990005,137.339996,133.509995,136.690002,135.852509,124486200
503,2020-12-29,138.050003,138.789993,134.339996,134.869995,134.043640,121047300


In [3]:
# create a function to calculate the SMA & EMA
# create the simple moving average
def SMA(df,period = 30,column = "Close"):
    return df[column].rolling(window= period).mean()

# create the exponential moving average
def EMA(df,period = 20,column = "Close"):
    return df[column].ewm(span= period,adjust = False).mean()

In [4]:
# calculate the Moving Average Convergence/Divergence(MACD)
def MACD(df,period_long = 26,period_short = 20,signal_line = 9,column= "Close"):
    #calculate the short ema
    ShortEMA = EMA(df,period_short,column = column)
    
    #calculate the long ema
    
    LongEMA = EMA(df,period_long, column = column)
    #calculate the MACD
    
    df["MACD"] = ShortEMA - LongEMA
    #calculate the signal line
    
    df["signal_line"]= EMA(df,signal_line,column = "MACD")
    
    return df
        




In [5]:
# create the function Relative Strength Index (RSI)

def RSI(df,period = 14,column = "Close"):
    delta = df[column].diff(1)
    delta = delta[1:]
    up = delta.copy()
    down = delta.copy()
    up[up<0]= 0
    down[down>0]= 0
    df["up"] = up
    df["down"]= down
    AVG_Gain = SMA(df,period,column = "up")
    AVG_LOSS = abs(SMA(df,period,column = "down"))
    RS = AVG_Gain/AVG_LOSS
    RSI = 100.0 - (100.0/1.0 + RS)
    
    df["RSI"]= RSI
    
    return df

In [6]:
# create / add to data set
MACD(df)
RSI(df)
df["SMA"] = SMA(df)
df["EMA"] = EMA(df)

In [7]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,MACD,signal_line,up,down,RSI,SMA,EMA
0,2018-12-31,39.632500,39.840000,39.119999,39.435001,38.282612,140014000,0.000000,0.000000,NaN,NaN,NaN,NaN,39.435001
1,2019-01-02,38.722500,39.712502,38.557499,39.480000,38.326294,148158800,0.000952,0.000190,0.044998,0.000000,NaN,NaN,39.439287
2,2019-01-03,35.994999,36.430000,35.500000,35.547501,34.508709,365248800,-0.081484,-0.016144,0.000000,-3.932499,NaN,NaN,39.068641
3,2019-01-04,36.132500,37.137501,35.950001,37.064999,35.981865,234428400,-0.117853,-0.036486,1.517498,0.000000,NaN,NaN,38.877818
4,2019-01-07,37.174999,37.207500,36.474998,36.982498,35.901768,219111200,-0.149236,-0.059036,0.000000,-0.082500,NaN,NaN,38.697311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,2020-12-23,132.160004,132.429993,130.779999,130.960007,130.157608,88223700,1.130197,0.892534,0.000000,-0.919998,-2.030850,122.093333,124.898504
501,2020-12-24,131.320007,133.460007,131.100006,131.970001,131.161407,54930100,1.196140,0.953255,1.009995,0.000000,-2.370947,122.509333,125.571980
502,2020-12-28,133.990005,137.339996,133.509995,136.690002,135.852509,124486200,1.342839,1.031172,4.720001,0.000000,-2.825109,123.092000,126.630839
503,2020-12-29,138.050003,138.789993,134.339996,134.869995,134.043640,121047300,1.417743,1.108486,0.000000,-1.820007,-2.177329,123.612333,127.415521


In [8]:
# create average true range(ATR)
def ATR(df,period = 14, column = "Close"):
    df["Range"] = df["High"] - df["Low"]
    df["atr_14"]= df["Range"].rolling(window = period).mean()
    return df

In [9]:
ATR(df)

,Date,Open,High,Low,Close,Adj Close,Volume,MACD,signal_line,up,down,RSI,SMA,EMA,Range,atr_14
0,2018-12-31,39.632500,39.840000,39.119999,39.435001,38.282612,140014000,0.000000,0.000000,NaN,NaN,NaN,NaN,39.435001,0.720001,NaN
1,2019-01-02,38.722500,39.712502,38.557499,39.480000,38.326294,148158800,0.000952,0.000190,0.044998,0.000000,NaN,NaN,39.439287,1.155003,NaN
2,2019-01-03,35.994999,36.430000,35.500000,35.547501,34.508709,365248800,-0.081484,-0.016144,0.000000,-3.932499,NaN,NaN,39.068641,0.930000,NaN
3,2019-01-04,36.132500,37.137501,35.950001,37.064999,35.981865,234428400,-0.117853,-0.036486,1.517498,0.000000,NaN,NaN,38.877818,1.187500,NaN
4,2019-01-07,37.174999,37.207500,36.474998,36.982498,35.901768,219111200,-0.149236,-0.059036,0.000000,-0.082500,NaN,NaN,38.697311,0.732502,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,2020-12-23,132.160004,132.429993,130.779999,130.960007,130.157608,88223700,1.130197,0.892534,0.000000,-0.919998,-2.030850,122.093333,124.898504,1.649994,2.829287
501,2020-12-24,131.320007,133.460007,131.100006,131.970001,131.161407,54930100,1.196140,0.953255,1.009995,0.000000,-2.370947,122.509333,125.571980,2.360001,2.902144
502,2020-12-28,133.990005,137.339996,133.509995,136.690002,135.852509,124486200,1.342839,1.031172,4.720001,0.000000,-2.825109,123.092000,126.630839,3.830002,3.010002
503,2020-12-29,138.050003,138.789993,134.339996,134.869995,134.043640,121047300,1.417743,1.108486,0.000000,-1.820007,-2.177329,123.612333,127.415521,4.449997,3.192858


In [12]:
# create the standard deviation(STD)
def STD(df,period = 20,column = "Close"):
    df["std"]= df[column].rolling(window= period).std()
    return df

In [13]:
STD(df)

,Date,Open,High,Low,Close,Adj Close,Volume,MACD,signal_line,up,down,RSI,SMA,EMA,Range,atr_14,std
0,2018-12-31,39.632500,39.840000,39.119999,39.435001,38.282612,140014000,0.000000,0.000000,NaN,NaN,NaN,NaN,39.435001,0.720001,NaN,NaN
1,2019-01-02,38.722500,39.712502,38.557499,39.480000,38.326294,148158800,0.000952,0.000190,0.044998,0.000000,NaN,NaN,39.439287,1.155003,NaN,NaN
2,2019-01-03,35.994999,36.430000,35.500000,35.547501,34.508709,365248800,-0.081484,-0.016144,0.000000,-3.932499,NaN,NaN,39.068641,0.930000,NaN,NaN
3,2019-01-04,36.132500,37.137501,35.950001,37.064999,35.981865,234428400,-0.117853,-0.036486,1.517498,0.000000,NaN,NaN,38.877818,1.187500,NaN,NaN
4,2019-01-07,37.174999,37.207500,36.474998,36.982498,35.901768,219111200,-0.149236,-0.059036,0.000000,-0.082500,NaN,NaN,38.697311,0.732502,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,2020-12-23,132.160004,132.429993,130.779999,130.960007,130.157608,88223700,1.130197,0.892534,0.000000,-0.919998,-2.030850,122.093333,124.898504,1.649994,2.829287,4.290210
501,2020-12-24,131.320007,133.460007,131.100006,131.970001,131.161407,54930100,1.196140,0.953255,1.009995,0.000000,-2.370947,122.509333,125.571980,2.360001,2.902144,4.190395
502,2020-12-28,133.990005,137.339996,133.509995,136.690002,135.852509,124486200,1.342839,1.031172,4.720001,0.000000,-2.825109,123.092000,126.630839,3.830002,3.010002,4.491252
503,2020-12-29,138.050003,138.789993,134.339996,134.869995,134.043640,121047300,1.417743,1.108486,0.000000,-1.820007,-2.177329,123.612333,127.415521,4.449997,3.192858,4.611354


In [18]:
# create the bollinger bands(BB)
def BollingerBands(df,period = 20,column = "Close"):
    df["sma_20"]= df[column].rolling(window = period).mean()
    df['upper_band']= df['sma_20'] + 2 *df['sma_20']
    df['lower_band']= df['sma_20'] - 2* df['sma_20']
    return df

In [19]:
BollingerBands(df)

,Date,Open,High,Low,Close,Adj Close,Volume,MACD,signal_line,up,...,RSI,SMA,EMA,Range,atr_14,std,'sma_20',sma_20,upper_band,lower_band
0,2018-12-31,39.632500,39.840000,39.119999,39.435001,38.282612,140014000,0.000000,0.000000,NaN,...,NaN,NaN,39.435001,0.720001,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-02,38.722500,39.712502,38.557499,39.480000,38.326294,148158800,0.000952,0.000190,0.044998,...,NaN,NaN,39.439287,1.155003,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-03,35.994999,36.430000,35.500000,35.547501,34.508709,365248800,-0.081484,-0.016144,0.000000,...,NaN,NaN,39.068641,0.930000,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-04,36.132500,37.137501,35.950001,37.064999,35.981865,234428400,-0.117853,-0.036486,1.517498,...,NaN,NaN,38.877818,1.187500,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-07,37.174999,37.207500,36.474998,36.982498,35.901768,219111200,-0.149236,-0.059036,0.000000,...,NaN,NaN,38.697311,0.732502,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,2020-12-23,132.160004,132.429993,130.779999,130.960007,130.157608,88223700,1.130197,0.892534,0.000000,...,-2.030850,122.093333,124.898504,1.649994,2.829287,4.290210,124.106000,124.106000,372.318000,-124.106000
501,2020-12-24,131.320007,133.460007,131.100006,131.970001,131.161407,54930100,1.196140,0.953255,1.009995,...,-2.370947,122.509333,125.571980,2.360001,2.902144,4.190395,124.903000,124.903000,374.709001,-124.903000
502,2020-12-28,133.990005,137.339996,133.509995,136.690002,135.852509,124486200,1.342839,1.031172,4.720001,...,-2.825109,123.092000,126.630839,3.830002,3.010002,4.491252,125.908001,125.908001,377.724002,-125.908001
503,2020-12-29,138.050003,138.789993,134.339996,134.869995,134.043640,121047300,1.417743,1.108486,0.000000,...,-2.177329,123.612333,127.415521,4.449997,3.192858,4.611354,126.699000,126.699000,380.097001,-126.699000


In [20]:
# create the stochastic  oscillator

def Stochastic(df,period = 14,column = 'Close'):
    df['14_period_low']= df["Low"].rolling(window= period).mean()
    df['14_period_high']= df["High"].rolling(window= period).mean()
    
    df['stochastic']= (df[column] - df['14_period_low'])/(df['14_period_high'] - df['14_period_low'] )
    return df

In [21]:
Stochastic(df)

,Date,Open,High,Low,Close,Adj Close,Volume,MACD,signal_line,up,...,Range,atr_14,std,'sma_20',sma_20,upper_band,lower_band,14_period_low,14_period_high,stochastic
0,2018-12-31,39.632500,39.840000,39.119999,39.435001,38.282612,140014000,0.000000,0.000000,NaN,...,0.720001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-02,38.722500,39.712502,38.557499,39.480000,38.326294,148158800,0.000952,0.000190,0.044998,...,1.155003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-03,35.994999,36.430000,35.500000,35.547501,34.508709,365248800,-0.081484,-0.016144,0.000000,...,0.930000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-04,36.132500,37.137501,35.950001,37.064999,35.981865,234428400,-0.117853,-0.036486,1.517498,...,1.187500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-07,37.174999,37.207500,36.474998,36.982498,35.901768,219111200,-0.149236,-0.059036,0.000000,...,0.732502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,2020-12-23,132.160004,132.429993,130.779999,130.960007,130.157608,88223700,1.130197,0.892534,0.000000,...,1.649994,2.829287,4.290210,124.106000,124.106000,372.318000,-124.106000,124.202141,127.031429,2.388540
501,2020-12-24,131.320007,133.460007,131.100006,131.970001,131.161407,54930100,1.196140,0.953255,1.009995,...,2.360001,2.902144,4.190395,124.903000,124.903000,374.709001,-124.903000,124.886428,127.788572,2.440807
502,2020-12-28,133.990005,137.339996,133.509995,136.690002,135.852509,124486200,1.342839,1.031172,4.720001,...,3.830002,3.010002,4.491252,125.908001,125.908001,377.724002,-125.908001,125.690713,128.700715,3.654247
503,2020-12-29,138.050003,138.789993,134.339996,134.869995,134.043640,121047300,1.417743,1.108486,0.000000,...,4.449997,3.192858,4.611354,126.699000,126.699000,380.097001,-126.699000,126.494284,129.687143,2.623264


In [24]:
df = df[["Date","Open",'High','Close','Volume','MACD','RSI','atr_14','SMA','EMA']]

In [25]:
df

,Date,Open,High,Close,Volume,MACD,RSI,atr_14,SMA,EMA
0,2018-12-31,39.632500,39.840000,39.435001,140014000,0.000000,NaN,NaN,NaN,39.435001
1,2019-01-02,38.722500,39.712502,39.480000,148158800,0.000952,NaN,NaN,NaN,39.439287
2,2019-01-03,35.994999,36.430000,35.547501,365248800,-0.081484,NaN,NaN,NaN,39.068641
3,2019-01-04,36.132500,37.137501,37.064999,234428400,-0.117853,NaN,NaN,NaN,38.877818
4,2019-01-07,37.174999,37.207500,36.982498,219111200,-0.149236,NaN,NaN,NaN,38.697311
...,...,...,...,...,...,...,...,...,...,...
500,2020-12-23,132.160004,132.429993,130.960007,88223700,1.130197,-2.030850,2.829287,122.093333,124.898504
501,2020-12-24,131.320007,133.460007,131.970001,54930100,1.196140,-2.370947,2.902144,122.509333,125.571980
502,2020-12-28,133.990005,137.339996,136.690002,124486200,1.342839,-2.825109,3.010002,123.092000,126.630839
503,2020-12-29,138.050003,138.789993,134.869995,121047300,1.417743,-2.177329,3.192858,123.612333,127.415521
